This notebook created some data for the DMS model via the REST API.

In [37]:
import requests
from rich.pretty import pprint
from rich import print_json
import json

# location of the DB (B=base URL for MongoDB, DB=database name)
B='http://localhost:8080'
DB='dms0'

# small wrapper for the REST API
def get(p,no_show=False,**kw):
    r=requests.get(f'{B}/{p}',params=kw)
    # pprint(r)
    data=json.loads(r.text)
    if not no_show: pprint(data)
    return data
def post(p,**kw):
    r=requests.post(f'{B}/{p}',json=kw)
    if not r.ok: raise RuntimeError(r.text)
    return json.loads(r.text)
def patch(p,**kw):
    r=requests.patch(f'{B}/{p}',json=kw)
    if not r.ok: raise RuntimeError(r.text)
    return json.loads(r.text)



The following dictionary is a `BeamState` which covers all types defined in the schema (and includes multiply-referenced objects).

Note:

* Any attribute can be omitted: won't be saved in the DB and won't be returned
* Numbers are given as
    - `{'value':56.3,'unit':'m'}` for scalars with unit
    - `{'value':[[1,2],[3,4],[5,6]],'unit':'m'}` for arrays with unit
    - `{'value':33}` or `33` for scalars without unit (similar for arrays without unit)
* Numbers are converted to the units specified in the schema (via `astropy.units`), and will be returned with that unit
* Strings are given as plain strings
* Bytes are given as strings (e.g. base64-encoded), and will be returned as such
* Relative links with dot-notation are used to reference previously defined objects (leading dots ascend)

In [41]:
dta={ # BeamState 
    "beam":{ # Beam
        "length": { "value": 2500, "unit":"mm" },
        "height": { "value": 20, "unit":"cm" },
        "density": { "value": 3.5, "unit":"g/cm3" },
        "cs":{ # CrossSection
            "rvePositions": {"value":[[1,2,3],[4,5,6]],"unit":"mm"},
            "rve":{  # ConcreteRVE
                "origin":{"value":[5,5,5],"unit":"mm"},
                "size":{ "value":[150,161,244],"unit":"um" },
                "ct":{ # CTScan
                    "id":"scan-000"
                },
                "materials":[
                    { # MaterialRecord
                        "name":"mat0",
                        "props":{"origin":"CZ","year":2018,"quality":"good"},
                    },
                    { # MaterialRecord
                        "name":"mat1",
                        "props":{"origin":"PL","year":2016,"project":"HTL-344PRP"},
                    }
                ]
            },        
        }
    },
    "cs": ".beam.cs", # relative link to the ../beam/cs object
    "npointz": 2,
    "csState":[
        { # CrossSectionState
            "eps_axial": { "value":344, "unit":"um/m" },
            "bendingMoment": { "value":869, "unit":"kN*m" },
            "rveStates":[ 
                { # ConcreteRVEState
                    "rve":"...beam.cs.rve", # rel 
                    "sigmaHom": { "value": 89.5, "unit":"MPa" }
                },
                { # ConcreteRVEState
                    "rve":"...beam.cs.rve", # rel 
                    "sigmaHom": { "value": 81.4, "unit":"MPa" }
                },
            ]
        },
        { # CrossSectionState
            "eps_axial": { "value":878, "unit":"um/m" },
            "bendingMoment": { "value":123, "unit":"kN*m" },
            "rveStates":[ 
                { # ConcreteRVEState
                    "rve":"...beam.cs.rve", # rel 
                    "sigmaHom": { "value": 55.6, "unit":"MPa" }
                },
            ]
        },

    ],

}


# Inserting (POST) and retrieving (GET)

* POST will create new object in the database, returning its ID
* GET wil lretrieve the object; in this case, we use
    * `meta=False` to strip metadata so that the result is more human-readable
    * `tracking=True` to use relative references for multiply-returned objects (instead of duplicating them)


In [54]:
ID=post('dms0/BeamState',**dta)
get(f'dms0/BeamState/{ID}',meta=False,tracking=True);

{
│   'beam': {
│   │   'length': {'value': 2.5, 'unit': 'm'},
│   │   'height': {'value': 0.2, 'unit': 'm'},
│   │   'density': {'value': 2999.9999999999995, 'unit': 'kg / m3'},
│   │   'cs': {
│   │   │   'rvePositions': {
│   │   │   │   'value': [
│   │   │   │   │   [0.0010000000474974513, 0.0020000000949949026, 0.003000000026077032],
│   │   │   │   │   [0.004000000189989805, 0.005000000353902578, 0.006000000052154064]
│   │   │   │   ],
│   │   │   │   'unit': 'm'
│   │   │   },
│   │   │   'rve': {
│   │   │   │   'origin': {
│   │   │   │   │   'value': [0.005000000353902578, 0.005000000353902578, 0.005000000353902578],
│   │   │   │   │   'unit': 'm'
│   │   │   │   },
│   │   │   │   'size': {'value': [150.0, 161.0, 244.0], 'unit': 'um'},
│   │   │   │   'ct': {'id': 'scan-000'},
│   │   │   │   'materials': [
│   │   │   │   │   {'name': 'mat0', 'props': {'origin': 'CZ', 'year': 2018, 'quality': 'good'}},
│   │   │   │   │   {'name': 'mat1', 'props': {'origin': 'PL', 'year': 2016, 'project': 'HTL-344PRP'}}
│   │   │   │   ]
│   │   │   }
│   │   }
│   },
│   'cs': 'beam.cs',
│   'npointz': {'value': 2},
│   'csState': [
│   │   {
│   │   │   'eps_axial': {'value': 344.0, 'unit': 'um / m'},
│   │   │   'bendingMoment': {'value': 869.0, 'unit': 'kN m'},
│   │   │   'rveStates': [
│   │   │   │   {'rve': '..beam.cs.rve', 'sigmaHom': {'value': 89.0, 'unit': 'MPa'}},
│   │   │   │   {'rve': '..beam.cs.rve', 'sigmaHom': {'value': 81.0, 'unit': 'MPa'}}
│   │   │   ]
│   │   },
│   │   {
│   │   │   'eps_axial': {'value': 878.0, 'unit': 'um / m'},
│   │   │   'bendingMoment': {'value': 123.0, 'unit': 'kN m'},
│   │   │   'rveStates': [{'rve': '..beam.cs.rve', 'sigmaHom': {'value': 55.0, 'unit': 'MPa'}}]
│   │   }
│   ]
}

In [55]:
# the same, but with metadata
get(f'dms0/BeamState/{ID}',tracking=True);

{
│   '_meta': {'id': '6358e51148169220e75baa28', 'type': 'BeamState'},
│   'beam': {
│   │   '_meta': {'id': '6358e51148169220e75baa22', 'type': 'Beam', 'parent': '6358e51148169220e75baa28'},
│   │   'length': {'value': 2.5, 'unit': 'm'},
│   │   'height': {'value': 0.2, 'unit': 'm'},
│   │   'density': {'value': 2999.9999999999995, 'unit': 'kg / m3'},
│   │   'cs': {
│   │   │   '_meta': {
│   │   │   │   'id': '6358e51148169220e75baa21',
│   │   │   │   'type': 'CrossSection',
│   │   │   │   'parent': '6358e51148169220e75baa22'
│   │   │   },
│   │   │   'rvePositions': {
│   │   │   │   'value': [
│   │   │   │   │   [0.0010000000474974513, 0.0020000000949949026, 0.003000000026077032],
│   │   │   │   │   [0.004000000189989805, 0.005000000353902578, 0.006000000052154064]
│   │   │   │   ],
│   │   │   │   'unit': 'm'
│   │   │   },
│   │   │   'rve': {
│   │   │   │   '_meta': {
│   │   │   │   │   'id': '6358e51148169220e75baa20',
│   │   │   │   │   'type': 'ConcreteRVE',
│   │   │   │   │   'parent': '6358e51148169220e75baa21'
│   │   │   │   },
│   │   │   │   'origin': {
│   │   │   │   │   'value': [0.005000000353902578, 0.005000000353902578, 0.005000000353902578],
│   │   │   │   │   'unit': 'm'
│   │   │   │   },
│   │   │   │   'size': {'value': [150.0, 161.0, 244.0], 'unit': 'um'},
│   │   │   │   'ct': {
│   │   │   │   │   '_meta': {
│   │   │   │   │   │   'id': '6358e51148169220e75baa1d',
│   │   │   │   │   │   'type': 'CTScan',
│   │   │   │   │   │   'parent': '6358e51148169220e75baa20'
│   │   │   │   │   },
│   │   │   │   │   'id': 'scan-000'
│   │   │   │   },
│   │   │   │   'materials': [
│   │   │   │   │   {
│   │   │   │   │   │   '_meta': {
│   │   │   │   │   │   │   'id': '6358e51148169220e75baa1e',
│   │   │   │   │   │   │   'type': 'MaterialRecord',
│   │   │   │   │   │   │   'parent': '6358e51148169220e75baa20'
│   │   │   │   │   │   },
│   │   │   │   │   │   'name': 'mat0',
│   │   │   │   │   │   'props': {'origin': 'CZ', 'year': 2018, 'quality': 'good'}
│   │   │   │   │   },
│   │   │   │   │   {
│   │   │   │   │   │   '_meta': {
│   │   │   │   │   │   │   'id': '6358e51148169220e75baa1f',
│   │   │   │   │   │   │   'type': 'MaterialRecord',
│   │   │   │   │   │   │   'parent': '6358e51148169220e75baa20'
│   │   │   │   │   │   },
│   │   │   │   │   │   'name': 'mat1',
│   │   │   │   │   │   'props': {'origin': 'PL', 'year': 2016, 'project': 'HTL-344PRP'}
│   │   │   │   │   }
│   │   │   │   ]
│   │   │   }
│   │   }
│   },
│   'cs': 'beam.cs',
│   'npointz': {'value': 2},
│   'csState': [
│   │   {
│   │   │   '_meta': {
│   │   │   │   'id': '6358e51148169220e75baa25',
│   │   │   │   'type': 'CrossSectionState',
│   │   │   │   'parent': '6358e51148169220e75baa28'
│   │   │   },
│   │   │   'eps_axial': {'value': 344.0, 'unit': 'um / m'},
│   │   │   'bendingMoment': {'value': 869.0, 'unit': 'kN m'},
│   │   │   'rveStates': [
│   │   │   │   {
│   │   │   │   │   '_meta': {
│   │   │   │   │   │   'id': '6358e51148169220e75baa23',
│   │   │   │   │   │   'type': 'ConcreteRVEState',
│   │   │   │   │   │   'parent': '6358e51148169220e75baa25'
│   │   │   │   │   },
│   │   │   │   │   'rve': '..beam.cs.rve',
│   │   │   │   │   'sigmaHom': {'value': 89.0, 'unit': 'MPa'}
│   │   │   │   },
│   │   │   │   {
│   │   │   │   │   '_meta': {
│   │   │   │   │   │   'id': '6358e51148169220e75baa24',
│   │   │   │   │   │   'type': 'ConcreteRVEState',
│   │   │   │   │   │   'parent': '6358e51148169220e75baa25'
│   │   │   │   │   },
│   │   │   │   │   'rve': '..beam.cs.rve',
│   │   │   │   │   'sigmaHom': {'value': 81.0, 'unit': 'MPa'}
│   │   │   │   }
│   │   │   ]
│   │   },
│   │   {
│   │   │   '_meta': {
│   │   │   │   'id': '6358e51148169220e75baa27',
│   │   │   │   'type': 'CrossSectionState',
│   │   │   │   'parent': '6358e51148169220e75baa28'
│   │   │   },
│   │   │   'eps_axial': {'value': 878.0, 'unit': 'um / m'},
│   │   │   'bendingMoment': {'value': 123.0, '